# Convolutional Neural Networks


A convolutional neural network (CNN) is a neural network similar to the ones we have seen before, but in this case, it assumes that the input data is comprised of images. For this reason, it is possible to make some changes in the network architecture that allow the model to recognize certain elements in the images that a basic neural network wouldn't be able to.

CNNs are specifically designed to identify borders, lines, textures or shapes in images, and how these components should be distributed in order to classify an image as a certain object. For instance, CNNs can identify the parts of a face and take into account how they are distributed in order to classify the image as a face or not.

In few words, these networks perform the following steps:

- First layers: they are in charge of recognizing basic elements and patterns.
- Intermediate/Final layers: they learn more complex patterns derived from the basic patterns learnt in previous layers


## Convolution


The main difference between densely connected layers (the ones we have worked with until now) and layers specialized in convolution tasks is that the first ones find global patterns in the dataset, while the second ones try to find small, local patterns inside a part of the image such as lines, edges or color patterns. What is more interesting is that once the networks learns this pattern, it is able to recognize it in any other part of the image, while a densely connected networks would have to learn the pattern again if it appears in any other part of the image.

Convolutional layers usually work with 3D tensors called _feature maps_, with two spacial axes (_height_ and _width_) and a channel axis, also called _depth_. The channel axis is used to work with colors, so if we have RGB images, we would need two more extra axes. The datasets we will be working are in black and white, therefore we will not need them.

As we have mentioned before, convolutional layers work 'locally', meaning that only a subset of the first layer (input neurons associated to each pixel of the image) is connected to each neuron of the second one. In any case, subsets can overlap, so some input neurons can be connected to several neurons in the next layer.

Another interesting characteristic of the convolutional layer is that in order to connect each neuron of the layer to X neurons of the input layer (the subset), we need a bias $b$ and a $X x X$ weight matrix $W$, which is usually called filter/kernel. The value assigned to a neuron is then given by the sum of the element-wise multiplication of the filter and the X input neurons of the subset associated to it (if we take into account all the output neurons, this is essentially a convolution: https://stats.stackexchange.com/questions/335321/in-a-convolutional-neural-network-cnn-when-convolving-the-image-is-the-opera).  This filter is the same for all of the neurons, so the number of parameters that constitute our neural networks decreases drastically to, as for a densely connected neural network we would need a filter for each of the input neurons .   

The disadvantage of using one filter is that it only allows us to detect one characteristic of an image. In order to find more, it is possible to add more filters. Since the reduction in parameters is inmense in comparison to a densely connected network, it is still possible to do this and have a model with less parameters.

For a better explanation of the subject, please check  https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53 

## Pooling

Now that we have explained what convolutional layers are, we might talk a bit about _pooling layers_, which usually come right after convolutional layers in the model. In essence, pooling layers gather information from convolutional layers and create a condensed version of it. The most common ways of pooling are _max pooling_ and _average pooling_. As you can see in the image below, max pooling consists in grouping the input values and passing the maximum value of each group to the next layer, while average pooling takes the average value of each group. 


<img src="https://qph.fs.quoracdn.net/main-qimg-939c3123c48e27301f1a89c0a299dca8" width="400" height="400" />


It is also important to notice that for each convolutional filter we have, we end up with pooling filter.

## Building a Convolutional Neural Network

It is time to get our hands dirty and build our first convolutional neural network. In this case, we will work with 5x5 convolutional windows and 2x2 pooling windows. For the convolutional ones, we will have a _stride_ of 1 in each dimension, which means that the 5x5 window moving around the image is horizontally/vertically displaced one column/row to create the next output (please have a look at image below).


<img src="https://adeshpande3.github.io/assets/Stride1.png" width="400" height="400" />

In this case, we will not use _padding_ , which, in essence, means surrounding the image by zeros in order to not lose information on the edges of the image . As you might guess with the picture above, when the window moves throughout the image, the pixels in the borders are covered less times than the ones in the middle, so there is loss of information and dimensionality. If we add sufficient zeros rows and columns to the borders, this problem is solved, and the output matrix would have the same size as the input matrix. For a really nice explanation of padding, please check https://www.machinecurve.com/index.php/2020/02/07/what-is-padding-in-a-neural-network/ or have a look at p.170-172 from _Python Deep Learning_ (link in References, in Spanish).


Now that we have explained these details, we can proceed with the code:


In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

In [4]:
model = Sequential()
model.add(Conv2D(32,(5,5), activation="relu", #convolutional layer of 32 filters and 5x5 windows. Activation function is relu
         input_shape=(28,28,1))) #we already specify input shape. Images are black and white, so we only have one color channel
model.add(MaxPooling2D(2,2)) #pooling with 2x2 window

# https://keras.io/api/layers/convolution_layers/convolution2d/

Let's have a look at the architecture of the model. We will see that we have an output shape of (24,24,32) for the convolutional layer (32 filters, 5x5 window in 28x28 image with a stride of (1,1) leaves space for 24 movements in each direction) and a pooling layer with a (12,12,32) shape (2x2 windows, it divides the 24x24 convolutional layer into two in each dimension).

On the other hand, for each filter we need 25 parameters (weight matrix _W_) plus the bias _b_. Since we have 32 filters, we end up with $32*26 = 832$ parameters. Bear in mind that the pooling layers do not require any extra parameters, as it only requires a mathematical operation to find the maximum in each 2x2 window).

In [5]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
Total params: 832
Trainable params: 832
Non-trainable params: 0
_________________________________________________________________


Now that we have seen the most basic CNN and learnt where the parameters come from, we can build a more complex network (although still very simple). The model will consist on two different convolutional+pooling layers with a final densely connected layer which is activated by a softmax function:

In [7]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
#the rest has been imported already

In [11]:
model = Sequential()
model.add(Conv2D(32,(5,5), activation="relu", #convolutional layer of 32 filters and 5x5 windows. Activation function is relu
         input_shape=(28,28,1))) #we already specify input shape. Images are black and white, so we only have one color channel
model.add(MaxPooling2D(2,2)) #pooling with 2x2 window
model.add(Conv2D(64,(5,5), activation="relu")) #remember that we only need to specify the input shape in the first layer
model.add(MaxPooling2D(2,2)) #pooling with 2x2 window
model.add(Flatten()) #remember that we need to flatten the 3D tensor to 1D in order to use a densely connected neural network
model.add(Dense(10,activation="softmax"))

Let's have a look at the summary:

In [12]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10250     
Total params: 62,346
Trainable params: 62,346
Non-trainable params: 0
__________________________________________________

We can summarize all we have done in a simple image:



<img src="https://miro.medium.com/max/3288/1*uAeANQIOQPqWZnnuH-VEyw.jpeg" width="400" height="400" />

## References


#### Literature


_Python Deep Learning_ , by Jordi Torres (https://www.marcombo.com/python-deep-learning-9788426728289/)

#### Images





Pooling: https://www.quora.com/What-is-max-pooling-in-convolutional-neural-networks

Stride: https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/

Convolutional Neural Network: https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53
